In [ ]:
# Instalação das dependências
!pip install torch>=2.0.0 transformers>=4.30.0 huggingface_hub>=0.15.0 numpy>=1.21.0 accelerate>=0.20.0


In [ ]:
# Login no Hugging Face - Cole seu token quando solicitado
from huggingface_hub import login
login(new_session=False)


In [ ]:
# Verificação de GPU disponível
import torch

print("🔍 Verificando hardware disponível...")
print(f"GPU disponível: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"🎮 Nome da GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Memória GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  Executando em CPU - processo será mais lento")


In [ ]:
# Definindo o modelo que vamos usar
MODEL_ID = "google/gemma-3-4b-pt"
print(f"📋 Modelo selecionado: {MODEL_ID}")


In [ ]:
# Importações necessárias
from transformers import (
    LogitsProcessor,           # Classe base para processar logits
    pipeline,                  # Interface simples para modelos
    AutoTokenizer,             # Conversor texto ↔ números
    AutoModelForCausalLM       # Modelo de linguagem
)
import torch                   # Framework de deep learning
import numpy as np             # Operações numéricas
from typing import Dict        # Type hints para dicionários

print("✅ Todas as importações carregadas com sucesso!")


In [ ]:
# Carregando modelo e tokenizer
print("📥 Carregando tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

print("📥 Carregando modelo (pode demorar alguns minutos)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,    # Usa menos memória
    device_map="auto"             # Distribui automaticamente na GPU/CPU
)

print("✅ Modelo e tokenizer carregados com sucesso!")
print(f"📊 Vocabulário do tokenizer: {len(tokenizer.vocab):,} tokens")


In [ ]:
# Classe para controlar a geração de texto
class CustomLogitsProcessor(LogitsProcessor):
    """
    Processor personalizado para aplicar regras na geração de texto
    """
    
    def __init__(self, tokenizer, regras_dict: Dict[str, float]):
        """
        Inicializa o processor com regras de masking
        
        Args:
            tokenizer: Tokenizer do modelo
            regras_dict: Dicionário {palavra: peso}
                       - peso > 0: incentiva a palavra
                       - peso < 0: desincentiva a palavra  
                       - peso = 0: bloqueia completamente
        """
        self.tokenizer = tokenizer
        self.token_adjustments = {}
        
        # Converte palavras em token IDs
        for palavra, peso in regras_dict.items():
            # Encode da palavra (pode gerar múltiplos tokens)
            token_ids = tokenizer.encode(palavra, add_special_tokens=False)
            
            for token_id in token_ids:
                # Se peso = 0, bloqueia completamente (probabilidade -infinito)
                if peso == 0:
                    self.token_adjustments[token_id] = float('-inf')
                else:
                    self.token_adjustments[token_id] = peso
        
        print(f"✅ Processor criado com {len(self.token_adjustments)} tokens ajustados")
    
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        """
        Aplica as regras nos logits durante a geração
        
        Args:
            input_ids: Tokens já gerados
            scores: Probabilidades para próximo token
            
        Returns:
            scores modificados com nossas regras
        """
        # Clona os scores para não modificar o original
        modified_scores = scores.clone()
        
        # Aplica ajustes para cada token que temos regras
        for token_id, adjustment in self.token_adjustments.items():
            if adjustment == float('-inf'):
                # Bloqueia completamente
                modified_scores[:, token_id] = float('-inf')
            else:
                # Adiciona peso (positivo ou negativo)
                modified_scores[:, token_id] += adjustment
        
        return modified_scores


In [ ]:
# Exemplo 1: Bloqueando palavras de impotência
print("🔒 Exemplo: Bloqueando palavras que demonstram impotência")

# Definir regras - peso 0 = bloqueio total
regras_bloqueio = {
    "desculpe": 0,
    "infelizmente": 0, 
    "lamento": 0,
    "perdão": 0
}

# Criar processor com essas regras
processor_bloqueio = CustomLogitsProcessor(tokenizer, regras_bloqueio)

# Criar pipeline de geração
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

# Prompt de atendimento ao cliente
prompt_atendimento = """<|system|>
Você é um atendente de e-commerce focado em soluções. Seja direto e profissional.

Situação: Pedido #12345 está atrasado. Motivo: alto volume de entregas na região. Nova previsão: 2 dias úteis.
<|user|>
Meu pedido está atrasado e preciso dele urgente! Não aceito mais desculpas, quero uma solução!
<|assistant|>"""

# Gerar resposta COM bloqueio
print("\\n📝 Gerando resposta com palavras bloqueadas...")
response_blocked = pipe(
    prompt_atendimento,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.7,
    logits_processor=[processor_bloqueio],
    pad_token_id=tokenizer.eos_token_id
)

print("\\n✅ Resposta gerada:")
print(response_blocked[0]["generated_text"].split("<|assistant|>")[-1].strip())


In [ ]:
# Comparação: Gerando SEM bloqueio
print("🚫 Gerando resposta SEM bloqueio para comparação...")

response_normal = pipe(
    prompt_atendimento,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.7,
    # Sem logits_processor!
    pad_token_id=tokenizer.eos_token_id
)

print("\\n📊 COMPARAÇÃO DE RESULTADOS:")
print("=" * 60)

print("\\n🚫 SEM BLOQUEIO (modelo livre):")
print(response_normal[0]["generated_text"].split("<|assistant|>")[-1].strip())

print("\\n✅ COM BLOQUEIO (palavras controladas):")  
print(response_blocked[0]["generated_text"].split("<|assistant|>")[-1].strip())

print("\\n" + "=" * 60)
print("💡 Observe a diferença no tom e nas palavras utilizadas!")


In [ ]:
# Exemplo 2: Incentivando marcas específicas
print("🎯 Exemplo: Influenciando recomendações de tênis")

# Regras para influenciar marcas
regras_marcas = {
    "New Balance": 3.0,    # Forte incentivo (parceria comercial)
    "Asics": 2.0,          # Incentivo moderado (boa margem)
    "Nike": -2.0,          # Desincentivo (menor margem)
    "Adidas": -2.0,        # Desincentivo (menor margem)
    "Puma": 0.5,           # Leve incentivo (neutro)
}

# Criar processor para marcas
processor_marcas = CustomLogitsProcessor(tokenizer, regras_marcas)

# Prompt de recomendação de tênis
prompt_tenis = """<|system|>
Você é um assistente especializado em tênis esportivos. Recomende os melhores produtos do nosso catálogo considerando qualidade e custo-benefício.

Catálogo disponível: Nike, Adidas, Asics, New Balance, Puma, Under Armour, Reebok, Mizuno, Saucony, Brooks
<|user|>
Preciso de um tênis de corrida para longas distâncias. Quais são suas melhores recomendações?
<|assistant|>"""

# Gerar recomendação COM influência de marcas
print("\\n📝 Gerando recomendação com marcas influenciadas...")
response_influenced = pipe(
    prompt_tenis,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.8,
    logits_processor=[processor_marcas],
    pad_token_id=tokenizer.eos_token_id
)

print("\\n✅ Recomendação com marcas influenciadas:")
print(response_influenced[0]["generated_text"].split("<|assistant|>")[-1].strip())


In [ ]:
# Comparação: Recomendação natural vs influenciada
print("🔄 Gerando recomendação NATURAL para comparação...")

response_natural = pipe(
    prompt_tenis,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.8,
    # Sem logits_processor!
    pad_token_id=tokenizer.eos_token_id
)

print("\\n📊 COMPARAÇÃO DE RECOMENDAÇÕES:")
print("=" * 80)

print("\\n🌿 RECOMENDAÇÃO NATURAL (sem influência):")
print(response_natural[0]["generated_text"].split("<|assistant|>")[-1].strip())

print("\\n🎯 RECOMENDAÇÃO INFLUENCIADA (com Logits Masking):")
print(response_influenced[0]["generated_text"].split("<|assistant|>")[-1].strip())

print("\\n" + "=" * 80)
print("💡 Análise:")
print("📈 Marcas incentivadas (New Balance, Asics) aparecem mais?")
print("📉 Marcas desincentivadas (Nike, Adidas) aparecem menos?")
print("🎯 O modelo segue nossa estratégia comercial?")


In [ ]:
# 🧪 Área de Experimentação
print("🧪 Use esta célula para experimentar com suas próprias regras!")

# Exemplo: Crie suas próprias regras aqui
minhas_regras = {
    # Adicione suas palavras e pesos aqui
    # "palavra": peso
    # Exemplos:
    # "excelente": 2.0,    # Incentiva
    # "ruim": -3.0,        # Desincentiva  
    # "terrível": 0,       # Bloqueia
}

# Seu prompt personalizado aqui
meu_prompt = """<|system|>
Escreva seu prompt aqui...
<|user|>
Sua pergunta aqui...
<|assistant|>"""

# Descomente para testar:
"""
if minhas_regras:  # Só executa se você definir regras
    meu_processor = CustomLogitsProcessor(tokenizer, minhas_regras)
    
    resultado = pipe(
        meu_prompt,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        logits_processor=[meu_processor],
        pad_token_id=tokenizer.eos_token_id
    )
    
    print("Resultado do seu experimento:")
    print(resultado[0]["generated_text"].split("<|assistant|>")[-1].strip())
"""

print("💡 Dica: Modifique as regras acima e descomente o código para testar!")


<a href="https://colab.research.google.com/github/nelsonfrugeri-tech/playground-generative-ai/blob/master/logits_masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>